In [1]:
import utils.dlc_bci as bci
import numpy as np
import matplotlib.pyplot as plt
import torch

In [2]:
train_input , train_target = bci.load(root='../../data_bci', one_khz = True)
test_input , test_target = bci.load ( root = '../../data_bci', train = False, one_khz = True)

In [ ]:
import tensorly as tl

In [ ]:
from tensorly.decomposition import parafac, tucker

In [ ]:
def decompose_tucker(inputs, decomposed_channels=3):
    tucker_rank = [train_inputs.shape[0]+test_inputs.shape[0], train_inputs.shape[1], decomposed_channels]
    inputs = inputs.numpy()
    core, tucker_factors = tucker(inputs, ranks=tucker_rank, n_iter_max=100, init="svd", tol=1e-06, random_state=None, verbose=True)
    core = torch.Tensor(core)
    return core[:train_inputs.shape[0]], core[-test_inputs.shape[0]:]
train_inputs, test_inputs = decompose_tucker(inputs, decomposed_channels=16)

In [ ]:
class AddGaussianNoise(object):
    """Add gaussian noise to a numpy.ndarray (H x W x C)
    """

    def __init__(self, mean, sigma, random_state=np.random):
        self.sigma = sigma
        self.mean = mean
        self.random_state = random_state

    def __call__(self, image):
        if isinstance(self.sigma, collections.Sequence):
            sigma = random_num_generator(
                self.sigma, random_state=self.random_state)
        else:
            sigma = self.sigma
        if isinstance(self.mean, collections.Sequence):
            mean = random_num_generator(self.mean, random_state=self.random_state)
        else:
            mean = self.mean
        row, col, ch = image.shape
        gauss = self.random_state.normal(mean, sigma, (row, col, ch))
        image += torch.Tensor(gauss).float()
        return image
train_inputs = torch.cat((AddGaussianNoise(means, stds).__call__(train_inputs), train_inputs), 0)
train_targets = torch.cat((train_targets, train_targets), 0)